# Chapter 5 - Django Application Management

## Django settings.py for the Real World

In [2]:
import os
import socket


5.1. Django ALLOWED_HOSTS definition

In [ ]:
ALLOWED_HOSTS = [
    '.coffeehouse.com',
    '.bestcoffeehouse.com',
]

5.2. Django ADMINS and MANAGERS definition


In [ ]:
ADMINS = (
    ('Webmaster','webmaster@coffeehouse.com'), 
    ('Administrator','admin@coffeehouse.com')
)
MANAGERS = ADMINS

5.3. Django project structure deployed to /www/

```
+-/www/+
       |
       +--STORE--+
       |
       +---manage.py
       |
       +---coffeestatic--+
       |                 |
       |                 +-(Consolidated static resources)
       |
       +---coffeehouse--+
                        |
                        +-__init__.py
                        +-settings.py
                        +-urls.py
                        +-wsgi.py
                        |
                        +---templates---+
                                        +-app_base_template.html
                                        +-app_header_template.html
                                        +-app_footer_template.html
```


5.4. Django settings.py with absolute path values

In [ ]:
# Other configuration variables omitted for brevity
STATIC_ROOT = '/www/STORE/coffeestatic/'

# Other configuration variables omitted for brevity
TEMPLATES = [
    {
    'BACKEND': 'django.template.backends.django.DjangoTemplates',
    'DIRS': ['/www/STORE/coffeehouse/templates/',],
    }
]


5.5. Django settings.py with dynamically determined absolute path


In [ ]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
PROJECT_DIR = os.path.dirname(os.path.abspath(__file__))

# Other configuration variables omitted for brevity
STATIC_ROOT = '%s/coffeestatic/' % (BASE_DIR)

# Other configuration variables omitted for brevity
TEMPLATES = [
    {
    'BACKEND': 'django.template.backends.django.DjangoTemplates',
    'DIRS': ['%s/templates/'% (PROJECT_DIR),],
    }
]

5.6. Django settings.py with control variable with host name to load different sets of variables

In [ ]:
if socket.gethostname().startswith('live'):
    DJANGO_HOST = "production"
elif socket.gethostname().startswith('test'):
    DJANGO_HOST = "testing"
else:
    DJANGO_HOST = "development"

# Define general behavior variables for DJANGO_HOST and all others
if DJANGO_HOST == "production":
    DEBUG = False
    STATIC_URL = 'http://static.coffeehouse.com/'
else:
    DEBUG = True
    STATIC_URL = '/static/'

# Define DATABASES variable for DJANGO_HOST and all others
if DJANGO_HOST == "production":
    # Use mysql for live host
    DATABASES = {
        'default': {
            'NAME': 'housecoffee',
            'ENGINE': 'django.db.backends.mysql',
            'USER': 'coffee',
            'PASSWORD': 'secretpass'
        }
    }
else:
    # Use sqlite for non live host
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.sqlite3',
            'NAME': os.path.join(BASE_DIR, 'coffee.sqlite3'),
        }
    }

# Define EMAIL_BACKEND variable for DJANGO_HOST
if DJANGO_HOST == "production":
    # Output to SMTP server on DJANGO_HOST production
    EMAIL_BACKEND = 'django.core.mail.backends.smtp.EmailBackend'
elif DJANGO_HOST == "testing":
    # Nullify output on DJANGO_HOST test
    EMAIL_BACKEND = 'django.core.mail.backends.dummy.EmailBackend'
else:
    # Output to console for all others
    EMAIL_BACKEND = 'django.core.mail.backends.console.EmailBackend'

# Define CACHES variable for DJANGO_HOST production and all other hosts
if DJANGO_HOST == "production":
    # Set cache
    CACHES = {
        'default': {
            'BACKEND': 'django.core.cache.backends.memcached.MemcachedCache',
            'LOCATION': '127.0.0.1:11211',
            'TIMEOUT':'1800',
        }
    }
    CACHE_MIDDLEWARE_SECONDS = 1800
else:
    # No cache for all other hosts
    pass


5.7. Python configparser sample file production.cfg

In [ ]:
[general]
DEBUG: false
STATIC_URL: http://static.coffeehouse.com/

[databases]
NAME: housecoffee
ENGINE: django.db.backends.mysql
USER: coffee
PASSWORD: secretpass

[security]
SECRET_KEY: %%ea)cjy@v9(7!b(20gl+4-6iur28dy=tc4f$-zbm-v=!t

5.8. Django settings.py with configparser import


In [ ]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
PROJECT_DIR = os.path.dirname(os.path.abspath(__file__))

# Access configparser to load variable values
from django.utils.six.moves import configparser
config = configparser.SafeConfigParser(allow_no_value=True)

# Import socket to read host name
import socket
if socket.gethostname().startswith('live'):
    config.read('%s/production.cfg' % (PROJECT_DIR))
elif socket.gethostname().startswith('test'):
    config.read('%s/testing.cfg' % (PROJECT_DIR))
else:
    config.read('%s/development.cfg' % (PROJECT_DIR))

DEBUG = config.get('general', 'DEBUG')
STATIC_URL = config.get('general', 'STATIC_URL')
DATABASES = {
    'default': {
        'NAME': config.get('databases', 'NAME'),
        'ENGINE': config.get('databases', 'ENGINE'),
        'USER': config.get('databases', 'USER'),
        'PASSWORD': config.get('databases', 'PASSWORD')
    }
}
SECRET_KEY = config.get('security', 'SECRET_KEY')


5.9. Override DJANGO_SETTINGS_MODULE to load application variables from a file called testing.py and not the default settings.py


In [ ]:
!export DJANGO_SETTINGS_MODULE=coffeehouse.load_testing
!python manage.py runserver


## Set Up Static Web Page Resources - Images, CSS, JavaScript


5.10. Django app structure with static directories


```
+-<BASE_DIR_project_name>
|
+-manage.py
|
+-bootstrap-3.1.1-dist+
|                     +-bootstrap.min.js
|
+-jquery-1-11-1-dist+
|                   +jquery.min.js
|
+-jquery-ui-1.10.4+
|                 +jquery-ui.min.js
|
+-website-static-default+
|                       +-favicon.ico
|                       +-robots.txt
|
|
+---+-<PROJECT_DIR_project_name>
    |
    +-__init__.py
    +-settings.py
    +-urls.py
    +-wsgi.py
    |
    +-about(app)-+
    |            +-__init__.py
    |            +-models.py
    |            +-tests.py
    |            +-views.py
    |            +-static-+
    |                     |
    |                     +-about-+
    |                             +-img-+
    |                             |     +-logo.png
    |                             |
    |                             +-css-+
    |                                   +-custom.css
    +-stores(app)-+
                  +-__init__.py
                  +-models.py
                  +-tests.py
                  +-views.py
                  +-static-+
                           |
                           +-stores-+
                                    +-img-+
                                    |     +-coffee.gif
                                    |
                                    +-css-+
                                          +-custom.css
```

5.11. Django STATICFILES_DIR definition with namespaces in settings.py


In [ ]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
STATICFILES_DIRS = (
    '%s/website-static-default/'% (BASE_DIR), 
    ('bootstrap', '%s/bootstrap-3.1.1-dist/'% (BASE_DIR)), 
    ('jquery', '%s/jquery-1-11-1-dist/'% (BASE_DIR)), 
    ('jquery-ui', '%s/jquery-ui-1.10.4/'% (BASE_DIR)), 
)


5.12. Django visualization of static resources in apps and STATICFILES_DIRS

```
+-favicon.ico
+-robots.txt
|
+-jquery+
|       +jquery.min.js
|
+-jquery-ui+
|          +jquery-ui.min.js
|
+-bootstrap+
|          +-bootstrap.css
|
+-about-+
|       +-img-+
|       |     +-logo.png
|       |
|       +-css-+
|             +-custom.css
|
+-stores-+
         +-img-+
         |     +-coffee.gif
         |
         +-css-+
               +-custom.css
```

## Access Static Resources in Django Templates

5.13. Django {% static %} tag to reference static resources

In [ ]:
{% load static %}

# For static resource at about/img/logo.png
<img src="{% static 'about/img/logo.gif' %}">

# For static resource at bootstrap/bootstrap.css
<link href="{% static 'bootstrap/bootstrap.css' %}" rel="stylesheet">

# For static resource at jquery/jquery.min.js
<script src="{% static 'jquery/jquery.min.js' %}"></script>


5.14. Django collectstatic command to copy all static resources


In [ ]:
$python manage.py collectstatic

"""
You have requested to collect static files at the destination
location as specified in your settings:
/www/STORE/coffeestatic
This will overwrite existing files!
Are you sure you want to do this?
Type 'yes' to continue, or 'no' to cancel: yes
yes
Copying '/www/STORE/website-static-default/sitemap.xml'
Copying '/www/STORE/website-static-default/robots.txt'
Copying '/www/STORE/website-static-default/favicon.ico'
....
....
....
Copying '/www/STORE/coffeehouse/about/static/css/custom.css'
"""

## Django Logging

5.15. Default LOGGING in Django projects


In [1]:
LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'filters': {
        'require_debug_false': {
            '()': 'django.utils.log.RequireDebugFalse',
        },
        'require_debug_true': {
            '()': 'django.utils.log.RequireDebugTrue',
        },
    },
    'handlers': {
        'console': {
            'level': 'INFO',
            'filters': ['require_debug_true'],
            'class': 'logging.StreamHandler',
        },
        'null': {
            'class': 'logging.NullHandler',
        },
        'mail_admins': {
            'level': 'ERROR',
            'filters': ['require_debug_false'],
            'class': 'django.utils.log.AdminEmailHandler'
        }
    },
    'loggers': {
        'django': {
            'handlers': ['console'],
        },
        'django.request': {
            'handlers': ['mail_admins'],
            'level': 'ERROR',
            'propagate': False,
        },
        'django.security': {
            'handlers': ['mail_admins'],
            'level': 'ERROR',
            'propagate': False,
        },
        'py.warnings': {
            'handlers': ['console'],
        },
    }
}


5.16. Define loggers in a Python module


In [ ]:
import logging


stdlogger = logging.getLogger(__name__) # Standard instance of a logger with __name__ (the current package name)
dbalogger = logging.getLogger('dba') # Custom instance logging with explicit name


5.17. Define log messages in a Python module

In [ ]:
import logging


stdlogger = logging.getLogger(__name__) # Standard instance of a logger with __name__
dbalogger = logging.getLogger('dba') # Custom instance logging with explicit name


def index(request):
    stdlogger.debug("Entering index method")


def contactform(request):
    stdlogger.info("Call to contactform method")
    try:
        # Logic to handle store_id
        stdlogger.debug("Entering store_id conditional block")
    except Exception, e:
        stdlogger.exception(e)

    stdlogger.info("Starting search on DB")
    try:
        # Logging to search db
        stdlogger.info("About to search db")
    except Exception, e:
        stdlogger.error("Error in searchdb method")
        dbalogger.error("Error in searchdb method, stack %s" % (e))



5.18. Custom LOGGING Django configuration


In [ ]:
LOGGING = {
    'version': 1,
    'disable_existing_loggers': True,
    'filters': {
        'require_debug_false': {
            '()': 'django.utils.log.RequireDebugFalse',
        },
        'require_debug_true': {
            '()': 'django.utils.log.RequireDebugTrue',
        },
    },
    'formatters': {
        'simple': {
            'format': '[%(asctime)s] %(levelname)s %(message)s',
            'datefmt': '%Y-%m-%d %H:%M:%S'
        },
        'verbose': {
            'format': '[%(asctime)s] %(levelname)s [%(name)s.%(funcName)s:%(lineno)d] 
            %(message)s',
            'datefmt': '%Y-%m-%d %H:%M:%S'
        },
    },
    'handlers': {
        'console': {
            'level': 'DEBUG',
            'filters': ['require_debug_true'],
            'class': 'logging.StreamHandler',
            'formatter': 'simple'
        },
        'development_logfile': {
            'level': 'DEBUG',
            'filters': ['require_debug_true'],
            'class': 'logging.FileHandler',
            'filename': '/tmp/django_dev.log',
            'formatter': 'verbose'
        },
        'production_logfile': {
            'level': 'ERROR',
            'filters': ['require_debug_false'],
            'class': 'logging.handlers.RotatingFileHandler',
            'filename': '/var/log/django/django_production.log',
            'maxBytes' : 1024*1024*100, # 100MB
            'backupCount' : 5,
            'formatter': 'simple'
        },
        'dba_logfile': {
            'level': 'DEBUG',
            'filters': ['require_debug_false','require_debug_true'],
            'class': 'logging.handlers.WatchedFileHandler',
            'filename': '/var/log/dba/django_dba.log',
            'formatter': 'simple'
        },
    },
    'root': {
        'level': 'DEBUG',
        'handlers': ['console'],
    },
    'loggers': {
        'coffeehouse': {
            'handlers': ['development_logfile','production_logfile'],
        },
        'dba': {
            'handlers': ['dba_logfile'],
        },
        'django': {
            'handlers': ['development_logfile','production_logfile'],
        },
        'py.warnings': {
            'handlers': ['development_logfile'],
        },
    }
}